In [1]:
import os
from keras.applications.resnet import ResNet50
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D



In [2]:
os.chdir('../')

In [3]:
model=ResNet50()
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [4]:
model.output

<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'predictions')>

In [5]:
from  pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size:list
    params_include_top:bool
    params_weights:str



In [6]:
from src.Fashion_Recommednation_System.constants import *
from src.Fashion_Recommednation_System.utils.common import read_yaml,create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_prepare_base_model_config(self) ->PrepareBaseModelConfig:
        config=self.config.prepare_base_model
        create_directories([config.root_dir])

        prepare_base_model_config=PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS

        )
        return prepare_base_model_config


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf


In [9]:
class PrepareBaseModel:
    def __init__ (self,config:PrepareBaseModelConfig):
        self.config=config
    def base_model(self):
        self.model=tf.keras.applications.ResNet50(
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top,
            weights=self.config.params_weights
        )
        self.save_model(path=self.config.base_model_path,model=self.model)
        
    def prepare_full_model(self):
        base_model=self.model
        base_model.trainable=False
        feature_extractor=Sequential()
        feature_extractor.add(base_model)
        feature_extractor.add(GlobalAveragePooling2D())
        feature_extractor.summary()
        self.save_model(path=self.config.updated_base_model_path, model=feature_extractor)

        return feature_extractor
    
    
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
    
    


        


In [10]:
try:
    config=ConfigurationManager()
    prepare_model_config=config.get_prepare_base_model_config()
    prepare_model=PrepareBaseModel(config=prepare_model_config)
    prepare_model.base_model()
    prepare_model.prepare_full_model()
   
except Exception as e:
    raise e

[2024-01-27 15:10:28,690: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-27 15:10:28,693: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-27 15:10:28,694: INFO: common: created directory at: artifacts]
[2024-01-27 15:10:28,695: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-01-27 15:10:30,120: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                           

value that i want in the otherfunction


In [18]:
read_yaml('config\\config.yaml')

EnsureError: Argument path_to_yaml of type <class 'str'> to <function read_yaml at 0x000001F42B1C57E0> does not match annotation type <class 'pathlib.Path'>